# Calculate SPG from CBS html



In [32]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression

import os
import sys

sys.path.append(r'C:\Users\lenha\Repos\phi-utils')

# N.B. this was has not been merged to main in that repo....
from philosofool.data_sources.utils import read_yml
#os.path.exists(os.path.norm('C:Users/lenha/Repos/phi-utils'))
standings_path = os.path.join(read_yml('local/config.yml')['paths']['local_data'], r'data\cbs_2021_standings.html')
from fantasy_baseball_draft.spg import spgs_from_standings_html as calc_spg

calc_spg(standings_path)

{'ba': 0.01151037472441089,
 'hr': 0.09595837983264877,
 'r': 0.0494239463865728,
 'rbi': 0.05261095031355526,
 'sb': 0.1478987411520366,
 'era': -0.0011116090409131923,
 'k': 0.033590558175417236,
 's': 0.1334702776908557,
 'w': 0.42727358758398803,
 'whip': -0.006384330680245498}